### Adding shapeless.

In [1]:
load.ivy("com.chuusai" %% "shapeless" % "2.2.2")

In [2]:
import shapeless._

import shapeless._

### Coproduct type with repeated types

In [3]:
type C1 = Int :+: Int :+: String :+: CNil

defined type C1

### Some instances of it

In [4]:
val c11: C1 = Inl(1)
val c12: C1 = Inr(Inl(2))
val c13: C1 = Inr(Inr(Inl("a")))

c11: C1 = 1
c12: C1 = 2
c13: C1 = a

### Check: embedded in its own type, a coproduct doesn't change

In [5]:
c11.embed[C1] == c11
c12.embed[C1] == c12
c13.embed[C1] == c13

res4_0: Boolean = true
res4_1: Boolean = true
res4_2: Boolean = true

### Defining `Basis0`, using `Remove` instead of `RemoveLast`

In [6]:
  trait Basis0[Super <: Coproduct, Sub <: Coproduct] extends DepFn1[Super] with Serializable {
    type Rest <: Coproduct
    type Out = Either[Rest, Sub]
    def inverse(e: Either[Rest, Sub]): Super
  }

  object Basis0 {
    import shapeless.ops.coproduct._
      
    type Aux[Super <: Coproduct, Sub <: Coproduct, Rest0 <: Coproduct] =
      Basis0[Super, Sub] { type Rest = Rest0 }

    def apply[Super <: Coproduct, Sub <: Coproduct](implicit basis: Basis0[Super, Sub]): Aux[Super, Sub, basis.Rest] =
      basis

    implicit def cnilBasis[Super <: Coproduct]: Aux[Super, CNil, Super] = new Basis0[Super, CNil] {
      type Rest = Super
      def apply(s: Super) = Left(s)
      def inverse(e: Either[Rest, CNil]) = e.left.get // No CNil exists, so e cannot be a Right
    }

    implicit def cconsBasis[Super <: Coproduct, H, T <: Coproduct, TRest <: Coproduct](implicit
      tailBasis: Basis0.Aux[Super, T, TRest],
      remove: Remove[TRest, H]
    ): Aux[Super, H :+: T, remove.Rest] = new Basis0[Super, H :+: T] {
      type Rest = remove.Rest

      def apply(s: Super) = tailBasis(s) match {
        case Left(r)  => remove(r) match {
          case Left(h)  => Right(Inl(h))
          case Right(r) => Left(r)
        }
        case Right(t) => Right(Inr(t))
      }

      def inverse(e: Either[Rest, H :+: T]) = e match {
        case Left(r)  => tailBasis.inverse(Left(remove.inverse(Right(r))))
        case Right(c) => c match {
          case Inl(h)  => tailBasis.inverse(Left(remove.inverse(Left(h))))
          case Inr(t)  => tailBasis.inverse(Right(t))
        }
      }
    }
  }


defined trait Basis0
defined object Basis0

### Define `embed0` method on coproducts using our `Basis0`

In [7]:
implicit class CoproductExtraOps[C <: Coproduct](c: C) {
  def embed0[Super <: Coproduct](implicit basis: Basis0[Super, C]): Super =
    basis.inverse(Right(c))
}

defined class CoproductExtraOps

### The check above doesn't pass with `Basis0` / `embed0`

In [8]:
c11.embed0[C1] == c11
c12.embed0[C1] == c12
c13.embed0[C1] == c13

res7_0: Boolean = false
res7_1: Boolean = false
res7_2: Boolean = true